<a href="https://colab.research.google.com/github/jh4581/resume-projects-portfolio.github.io/blob/main/Maven_Advanced_LLM_Module_2a_Mistral_7b_instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation of Libraries

If you use our code, please cite:

@misc{2024<br>
  title = {Mistral -7b hosting},<br>
  author = {Hamza Farooq, Darshil Modi, Kanwal Mehreen, Nazila Shafiei},<br>
  keywords = {LLM Hosting, Mistral-7b},<br>
  year = {2024},<br>
  copyright = {MIT, non-exclusive license}<br>
}


In [1]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
!pip install git+https://github.com/huggingface/transformers torch accelerate bitsandbytes langchain

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-pwcngh_4
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-pwcngh_4
  Resolved https://github.com/huggingface/transformers to commit aec1ca3a588bc6c65f7886e3d3eaa74901a6356f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install pyngrok --quiet
!pip install fastapi nest-asyncio --quiet
!pip install uvicorn --quiet
!pip install langchain-community --quiet
!pip install -U langchain-huggingface --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.7/307.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.4/341.4 kB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 44.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

# Setup

Get you Hugging Face Token, [here](https://huggingface.co/settings/tokens)

In [ ]:
from google.colab import userdata
HUGGING_FACE_TOKEN = userdata.get('HUGGING_FACE_TOKEN')

In [ ]:
!huggingface-cli login --token $HUGGING_FACE_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
# Import libraries
from transformers import AutoModelForCausalLM, AutoTokenizer , BitsAndBytesConfig
import transformers
import torch
from langchain_huggingface import HuggingFacePipeline
from langchain_core.output_parsers import StrOutputParser
from langchain.prompts import PromptTemplate
from threading import Thread

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
!nvidia-smi

Fri Jul 12 12:09:21 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8               9W /  70W |      3MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import textwrap

def wrap_text(text, width=90): #preserve_newlines
    # Split the input text into lines based on newline characters
    lines = text.split('\n')

    # Wrap each line individually
    wrapped_lines = [textwrap.fill(line, width=width) for line in lines]

    # Join the wrapped lines back together using newline characters
    wrapped_text = '\n'.join(wrapped_lines)

    return wrapped_text

# Basic Prompt

**Note:** If you are on windows then bitsandbytes won't work. You can counter this with bitsandbytes-windows but that is and older version that only supports 8-bit quantization.

In [ ]:
# Define the model ID for the desired model
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Define the quantization configuration for the model
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16
)

# Load the model using the model ID and quantization configuration
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=quantization_config, device_map='auto')

# Load the tokenizer associated with the model
tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

In [ ]:
vegetarian_recipe_prompt = """### Instruction: Act as a gourmet chef.
what is the best chicken dish for large groups

 ### Answer:
 """
prompt_template = """
### [INST]
Instruction: Act as a gourmet chef.


### QUESTION:
what is the best chicken dish for large groups

[/INST]
 """

# Encode the instruction using the tokenizer
encoded_instruction = tokenizer(prompt_template, return_tensors="pt", add_special_tokens=True)

# Move the encoded instruction to the appropriate device (e.g., GPU)
model_inputs = encoded_instruction.to(device)

# Generate text based on the model inputs
generated_ids = model.generate(**model_inputs, max_new_tokens=1000, do_sample=True, pad_token_id=tokenizer.eos_token_id)

# Decode the generated text
decoded = tokenizer.batch_decode(generated_ids)

# Print the decoded output
print(decoded)

["<s> \n### [INST]\nInstruction: Act as a gourmet chef.\n\n\n### QUESTION:\nwhat is the best chicken dish for large groups\n\n[/INST]\n  I would highly recommend serving roasted chicken for a large group gathering.\n  Roasting a whole chicken is a great choice as it allows the hostess to serve a complete and flavorful dish to guests.\n  Roasting brings out the tenderness and richness of the taste of the chicken.\n  Here's how to perfectly roast a whole chicken:\n  - Preheat the oven to 425°F.\n  - Rinse the chicken well and pat it dry.\n  - Season the inside and outside of the chicken with salt, pepper, and any other seasoning you prefer (such as rosemary, thyme, or garlic powder).\n  - Place the chicken breast-side down in a roasting pan.\n  - Roast for 1 hour and 20 minutes, or until the internal temperature reaches 165°F.\n  - Remove from the oven and let it rest for 20 minutes before carving and serving.\n  - Serve it with a side salad, roasted vegetables or a grain.\n\nAnother opt

In [ ]:
response = decoded[0].split("[/INST]")[1]
print((response.split('</s>')[0]).strip())

I would highly recommend serving roasted chicken for a large group gathering.
  Roasting a whole chicken is a great choice as it allows the hostess to serve a complete and flavorful dish to guests.
  Roasting brings out the tenderness and richness of the taste of the chicken.
  Here's how to perfectly roast a whole chicken:
  - Preheat the oven to 425°F.
  - Rinse the chicken well and pat it dry.
  - Season the inside and outside of the chicken with salt, pepper, and any other seasoning you prefer (such as rosemary, thyme, or garlic powder).
  - Place the chicken breast-side down in a roasting pan.
  - Roast for 1 hour and 20 minutes, or until the internal temperature reaches 165°F.
  - Remove from the oven and let it rest for 20 minutes before carving and serving.
  - Serve it with a side salad, roasted vegetables or a grain.

Another option is to make a buffet style dish with chicken, like chicken tikka masala or shawarma.
Chicken tikka masala is a spicy and delicious dish that is pe

# Creating LLM Chain to run a RAG prompt


In [ ]:

text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",  # Specify the task as text generation
    temperature=0.3,  # Temperature parameter for controlling randomness in sampling
    repetition_penalty=1.1,  # Repetition penalty parameter to avoid repeating tokens
    return_full_text=True,  # Flag to return full text instead of a list of generated tokens
    max_new_tokens=1000,  # Maximum number of tokens to generate
    do_sample=True  # Flag to use sampling during text generation
)

prompt_template = """
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable.

### CONTEXT:
{context}

### QUESTION:
{question}

[/INST]
 """

# Create HuggingFacePipeline object wrapping the text generation pipeline
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# Create prompt object from the prompt template with input variables as context and question
prompt = PromptTemplate(
    input_variables=["context", "question"],  # Specify input variables for the prompt
    template=prompt_template,  # Specify the template for the prompt
)

# Create language model chain (llm_chain) with HuggingFacePipeline and prompt
llm_chain = prompt | mistral_llm | StrOutputParser()


In [ ]:
prompt_base = """
### [INST]
Instruction: Act as a gourmet chef.


### QUESTION:
what is the best chicken dish for large groups

[/INST]
 """


In [ ]:
response=text_generation_pipeline(prompt_base)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [ ]:
response

[{'generated_text': '\n### [INST]\nInstruction: Act as a gourmet chef.\n\n\n### QUESTION:\nwhat is the best chicken dish for large groups\n\n[/INST]\n \nAs a gourmet chef, I would recommend roasting a whole chicken for a large group. This classic dish is both delicious and easy to prepare. To make it even more special, you can stuff the chicken with herbs, such as rosemary, thyme, and sage, and serve it with a side of roasted vegetables, mashed potatoes, or rice pilaf. If you want to add some extra flavor, you can also marinate the chicken in a mixture of lemon juice, olive oil, garlic, and herbs before roasting it in the oven. This will ensure that your guests are treated to a mouthwatering meal that they will love.'}]

In [ ]:
r=response[0].get('generated_text').split("[/INST]")[1].split('</s>')[0]


In [ ]:
print(r.strip())

As a gourmet chef, I would recommend roasting a whole chicken for a large group. This classic dish is both delicious and easy to prepare. To make it even more special, you can stuff the chicken with herbs, such as rosemary, thyme, and sage, and serve it with a side of roasted vegetables, mashed potatoes, or rice pilaf. If you want to add some extra flavor, you can also marinate the chicken in a mixture of lemon juice, olive oil, garlic, and herbs before roasting it in the oven. This will ensure that your guests are treated to a mouthwatering meal that they will love.


In [ ]:
# Input query to LLM
input_text = {
    "context": "Artificial Intelligence (AI) is the stream of data science that can predict future based on past trends!",
    "question": "What is AI ?"
}

# Generate and print LLM response
response = llm_chain.invoke(input_text)

wrapped_text = wrap_text(response)
print(f'The answer is: \n {wrapped_text}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The answer is: 
 
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an
answer easily understandable.

### CONTEXT:
Artificial Intelligence (AI) is the stream of data science that can predict future based
on past trends!

### QUESTION:
What is AI ?

[/INST]

AI, or Artificial Intelligence, refers to the field of computer science that focuses on
creating intelligent machines capable of performing tasks that would normally require
human intelligence, such as understanding natural language, recognizing patterns, and
making predictions. AI systems use algorithms and machine learning techniques to analyze
large amounts of data and make decisions based on that information.


In [ ]:
##############
question_rag = "What are the differences in work expectations between generations?"
##############


context_rag = """
'Generational differences in work expectations and values are a significant topic of discussion in organizations today. Specifically, the differences between baby boomers and millennials have garnered attention due to their coexistence in the workplace. These generational differences can impact management dynamics at various levels, including operational, team, and strategic levels. Understanding these differences and finding ways to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hierarchy and authority. They may prefer a structured work environment and clear guidelines.
- Preferences: Baby boomers may prefer face-to-face communication and value personal relationships in the workplace. They may also prioritize financial stability and job security.

Millennials:
- Work Values: Millennials often prioritize work-life balance, personal growth, and purpose-driven work. They seek opportunities for learning and development.
- Attitudes: Millennials may have a more collaborative and inclusive approach to work. They value flexibility, autonomy, and a sense of purpose in their roles.
- Preferences: Millennials are comfortable with technology and may prefer digital communication channels. They value feedback and recognition and seek a positive and engaging work environment.

2. Factors Contributing to Variations:
a) Socioeconomic Factors: Socioeconomic factors such as technological advancements, economic conditions, and societal changes can shape the expectations and values of different generations. For example, baby boomers experienced significant economic growth and stability, while millennials entered the workforce during times of economic uncertainty.
b) Cultural Influences: Cultural factors, including societal norms, values, and beliefs, can also contribute to generational differences. Each generation grows up in a unique cultural context that influences their perspectives on work.
c) Life Experiences: Different life experiences, such as major historical events or technological advancements, can shape the values and attitudes of each generation. For example, baby boomers may have experienced significant social and political changes, while millennials grew up in the digital age.

3. Practical Scenarios and Examples:
To illustrate these generational differences, consider the following scenarios:
- Baby boomers may prefer a structured work environment with clear hierarchies, while millennials may thrive in a more flexible and collaborative setting.
- Baby boomers may prioritize job security and financial stability, while millennials may prioritize personal growth and work-life balance.
- Baby boomers may prefer face-to-face communication, while millennials may be more comfortable with digital communication tools.

4. Navigating Generational Differences:
To create a harmonious and productive work environment, organizations can consider the following strategies:
- Foster open communication and create opportunities for intergenerational dialogue and understanding.
- Provide flexible work arrangements and opportunities for personal growth and development.
- Recognize and appreciate the diverse perspectives and strengths that each generation brings to the table.
- Implement mentorship or reverse-mentoring programs to facilitate knowledge sharing and collaboration between different generations.
- Create a culture of inclusivity and respect, valuing the contributions of all employees regardless of their generational background.

In summary, generational differences in work expectations and values exist between baby boomers and millennials. These differences can be attributed to various factors such as socioeconomic influences, cultural factors, and life experiences. Understanding and navigating these differences is crucial for organizations to create a harmonious and productive work environment. By fostering open communication, providing flexibility, and appreciating the diverse perspectives of each generation, organizations can effectively manage generational differences and leverage the strengths of their multi-generational workforce.'
"""

In [ ]:
# Input query to LLM
input_text = {
    "context": context_rag,
    "question": question_rag
}

# Generate and print LLM response
response = llm_chain.invoke(input_text)

wrapped_text = wrap_text(response)
print(f'The answer is: \n {wrapped_text}')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


The answer is: 
 
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an
answer easily understandable.

### CONTEXT:

'Generational differences in work expectations and values are a significant topic of
discussion in organizations today. Specifically, the differences between baby boomers and
millennials have garnered attention due to their coexistence in the workplace. These
generational differences can impact management dynamics at various levels, including
operational, team, and strategic levels. Understanding these differences and finding ways
to navigate them is crucial for creating a harmonious and productive work environment.

1. Contrasting Work Values, Attitudes, and Preferences:
Baby Boomers:
- Work Values: Baby boomers tend to prioritize loyalty, dedication, and hard work. They
often value stability and long-term commitment to their organizations.
- Attitudes: Baby boomers may have a more traditional approach to work, valuing hie

# Streaming Output & Metrics for Evaluation

In [ ]:
from transformers import TextIteratorStreamer
from threading import Thread
import time

first_token_time = 0
token_times = []


# Initialize a TextIteratorStreamer object for streaming text generation
streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

# Create a text generation pipeline using the Hugging Face transformers library
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    temperature=0.3,
    repetition_penalty=1.1,
    max_new_tokens=1000,
    do_sample=True,
    streamer=streamer  # Use the streamer for streaming text generation
)

prompt_template = """
### [INST]
Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable.

### CONTEXT:
{context}

### QUESTION:
{question}

[/INST]
 """

mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm_chain = prompt | mistral_llm | StrOutputParser()


# Input query to LLM
input_text = {
    "context": """"
    Act as a gourmet chef.
    I have a friend coming over who is a vegetarian.
    I want to impress my friend with a special vegetarian dish.
    """,
    "question": "Give me two options, along with the whole recipe for each."
}



In [ ]:
# Initialize variables for time measurements
start_time = time.time()

# Start a new thread to invoke the language model chain with the input text
thread = Thread(target=llm_chain.invoke, args=[input_text])
thread.start()

# Initialize a variable to store the model output
model_output = ""

# Iterate over the streamer to get the generated text in chunks
for i, new_text in enumerate(streamer):
    model_output += new_text
    print(new_text, end='')

    # Measure time for the first token
    if i == 0:
        first_token_time = time.time()
    # Measure time for each token
    token_times.append(time.time())

# Calculate end-to-end latency
end_time = time.time()
end_to_end_latency = end_time - start_time

# Calculate time to first token
ttft = first_token_time - start_time

# Calculate inter-token latency
itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

# Calculate throughput
throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

print("\nTime To First Token (TTFT):", ttft)
print("Inter-token latency (ITL):", itl)
print("End-to-end Latency:", end_to_end_latency)
print("Throughput:", throughput)


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


1. Vegan Lentil Shepherd's Pie

Ingredients:

* 2 tablespoons olive oil
* 1 onion, chopped
* 3 cloves garlic, minced
* 1 carrot, diced
* 1 celery stalk, diced
* 1 cup green lentils, rinsed and drained
* 4 cups vegetable broth
* 1 teaspoon dried thyme
* Salt and pepper to taste
* 1 cup frozen peas
* 1/4 cup nutritional yeast
* 2 tablespoons tomato paste
* 2 tablespoons soy sauce
* 1/4 cup vegan butter
* 1/2 cup all-purpose flour
* 2 cups unsweetened almond milk
* 1/4 teaspoon turmeric
* 1/4 teaspoon paprika
* 1/4 teaspoon cumin
* 1/4 teaspoon black pepper
* 2 tablespoons chopped fresh parsley

Instructions:

1. Preheat oven to 375°F (190°C).
2. In a large pot, heat olive oil over medium heat. Add onion, garlic, carrot, and celery, and cook until softened, about 5 minutes.
3. Stir in lentils, vegetable broth, thyme, salt, and pepper. Bring to a boil, then reduce heat and simmer for 20 minutes or until lentils are tender.
4. Stir in peas, nutritional yeast, tomato paste, soy sauce, vegan 

Let's create it into a function

In [ ]:
def streaming_inference(context, question):
    # Initialize a TextIteratorStreamer object for streaming text generation
    streamer = TextIteratorStreamer(tokenizer, timeout=10., skip_prompt=True, skip_special_tokens=True)

    # Create a text generation pipeline using the Hugging Face transformers library
    text_generation_pipeline = transformers.pipeline(
        model=model,
        tokenizer=tokenizer,
        task="text-generation",
        temperature=0.3,
        repetition_penalty=1.1,
        max_new_tokens=1000,
        do_sample= True,
        streamer=streamer  # Use the streamer for streaming text generation
    )

    prompt_template = """
    ### [INST]
    Instruction: I will ask you a QUESTION and give you a CONTEXT and you will respond with an answer easily understandable in bullet points.

    ### CONTEXT:
    {context}

    ### QUESTION:
    {question}

    [/INST]
    """

    mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

    prompt = PromptTemplate(
        input_variables=["context", "question"],
        template=prompt_template,
    )

    llm_chain = prompt | mistral_llm | StrOutputParser()

    input_text = {
        "context": context,
        "question": question
    }

    # Initialize variables for time measurements
    start_time = time.time()

    # Start a new thread to invoke the language model chain with the input text
    thread = Thread(target=llm_chain.invoke, args=[input_text])
    thread.start()

    # Initialize a variable to store the model output
    model_output = ""

    # Iterate over the streamer to get the generated text in chunks
    for i, new_text in enumerate(streamer):
        model_output += new_text
        print(new_text, end='')

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()
        # Measure time for each token
        token_times.append(time.time())

    # Calculate end-to-end latency
    end_time = time.time()
    end_to_end_latency = end_time - start_time

    # Calculate time to first token
    ttft = first_token_time - start_time

    # Calculate inter-token latency
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    # Calculate throughput
    throughput = len(tokenizer.encode(model_output)) / end_to_end_latency

    print("\nTime To First Token (TTFT):", ttft)
    print("Inter-token latency (ITL):", itl)
    print("End-to-end Latency:", end_to_end_latency)
    print("Throughput:", throughput)

    # Return the metrics
    return {
        "TTFT": ttft,
        "ITL": itl,
        "End-to-end Latency": end_to_end_latency,
        "Throughput": throughput
    }


In [ ]:
streaming_inference(context_rag,question_rag)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



   The differences in work expectations between generations include:

   - Baby boomers tend to prioritize loyalty, dedication, and hard work, while millennials prioritize work-life balance, personal growth, and purpose-driven work.
   - Baby boomers may have a more traditional approach to work, valuing hierarchy and authority, while millennials may have a more collaborative and inclusive approach to work.
   - Baby boomers may prefer face-to-face communication and value personal relationships in the workplace, while millennials are comfortable with technology and may prefer digital communication channels.
   - Baby boomers may prioritize financial stability and job security, while millennials may prioritize personal growth and work-life balance.

   These differences can be attributed to various factors such as socioeconomic influences, cultural factors, and life experiences. Organizations can navigate these differences by fostering open communication, providing flexibility, and appr

{'TTFT': 5.885674238204956,
 'ITL': 0.25101944495291606,
 'End-to-end Latency': 29.334325551986694,
 'Throughput': 7.124759000496103}

Time To First Token (TTFT): 5.885674238204956
Inter-token latency (ITL): 0.25101944495291606
End-to-end Latency: 29.334325551986694
Throughput: 7.124759000496103

# Setting Up a FastAPI Wrapper

In [ ]:
from fastapi import FastAPI, Request, BackgroundTasks
from fastapi.responses import StreamingResponse
from threading import Thread

app = FastAPI()

# Variables for time measurements
start_time = 0
first_token_time = 0
token_times = []

# Invoke the LLM chain using the input text
def invoke_llm_chain(input_text):
    llm_chain.invoke(input_text)


# Function to calculate metrics
def calculate_metrics(start_time, first_token_time, token_times, model_output):
    end_time = time.time()
    end_to_end_latency = end_time - start_time
    ttft = first_token_time - start_time
    itl = sum(x - y for x, y in zip(token_times[1:], token_times[:-1])) / (len(token_times) - 1)

    throughput = len(model_output) / end_to_end_latency
    return {
        "End-to-end Latency": end_to_end_latency,
        "Time To First Token (TTFT)": ttft,
        "Inter-token latency (ITL)": itl,
        "Throughput": throughput
    }

# Generate output text using the streamer
def generate_output(streamer):
    global start_time, first_token_time, token_times, model_output
    model_output = ""
    start_time = time.time()

    for i, new_text in enumerate(streamer):
        model_output += new_text

        # Measure time for the first token
        if i == 0:
            first_token_time = time.time()

        # Measure time for each token
        token_times.append(time.time())
        yield new_text

    metrics = calculate_metrics(start_time, first_token_time, token_times, model_output)
    print("Metrics:", metrics)
    return metrics


@app.get("/")
async def root():
    return {"message": "Hello, World!"}


@app.post("/mistral-inference")
async def mistral_inference(input_text: dict, background_tasks: BackgroundTasks):
    # Start a separate thread to run the LLM chain asynchronously
    thread = Thread(target=invoke_llm_chain, args=[input_text])
    thread.start()

    # Add the generate_output function to the background tasks with the streamer
    background_tasks.add_task(generate_output, streamer)

    return StreamingResponse(generate_output(streamer))


# Using Ngrok to Create Public URL

Sign up for your own key on Ngrok

In [ ]:
NGROK_KEY = userdata.get('NGROK_KEY')

In [ ]:
!ngrok config add-authtoken $NGROK_KEY

/bin/bash: line 1: ngrok: command not found


In [ ]:
import nest_asyncio
from pyngrok import ngrok
import uvicorn
ngrok.kill()

In [ ]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()

uvicorn.run(host="127.0.0.1", port=8000, app=app)
